# VQGAN+CLIP Simplificado



In [1]:
# Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

## Docker Volumenes

In [2]:
from extra import * 

MODELS_FOLDER = Path("/tf/models")
OUTPUTS_FOLDER= Path("/tf/outputs")
SRC_FOLDER= Path("/tf/outputs/src")

## Descargar modelos pre-entrenados:

In [3]:
from get_models import list_models, download_models
models = list_models(return_list=True)
print(models)

Models: 6
['vqgan_imagenet_f16_1024', 'vqgan_imagenet_f16_16384', 'coco', 'faceshq', 'wikiart', 'sflckr']


In [ ]:
download_models(models, output=MODELS_FOLDER)

	0 - downloading 'coco' ... 

In [16]:
# Selecciona un modelo
model = models[1] 

if (Path(MODELS_FOLDER)/ f"{model}.yaml").exists():
    print(f"{model} ya existe.")
else:
    print(f"Descargando: {model}")
    download_models(model, output=MODELS_FOLDER)

vqgan_imagenet_f16_16384 ya existe.


## Configuración

**Definir parámetros principales:**

In [17]:
#!ls $MODELS_FOLDER 

In [18]:
#Texto usado como Input para generar la imagen:
txt = 'Todo va a salir bien'
prompts = [txt]

# Tamaño de imagen
#size = (200, 200) #6129MiB de V-RAM
size = (340, 340) #8693MiB de V-RAM
#size = (450, 450) #10201MiB de V-RAM

# Numero de iteraciones en la creación de la imagen, a mayor el número de iteraciones mejor es el detalle en la imagen
iterations = 250

# Modelo a usar
# modelos descargados:
# !ls $MODELS_FOLDER 
model = 'vqgan_imagenet_f16_1024'

**Definir parámetros secundarios:**

In [19]:
config = {
    "init_image": None,
    "seed": None,
    "step_size": 0.2,
    "cutn": 64,
    "cut_pow": 1.,
    "display_freq": 5,
    "image_prompts": [],
    "noise_prompt_seeds": [],
    "noise_prompt_weights": [],
    "init_weight": 0.,
    "clip_model": 'ViT-B/32',
}

# SOBREESCRIBIR RESULTADOS
overwrite=True

## Generar frames del proceso resultante:

In [20]:
from generate_images import *

last_frame_index = generate_images(
    prompts=prompts,
    model=model, 
    outputs_folder=OUTPUTS_FOLDER,
    models_folder=MODELS_FOLDER,
    iterations=280,
    **config,
    overwrite=overwrite
)

Using device: cuda:0
Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Restored from /tf/models/vqgan_imagenet_f16_1024.ckpt


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
Training: 100%|██████████| 280/280 [01:56<00:00,  2.40it/s]


In [21]:
experiment_name = to_experiment_name(prompts)
experiment_folder = Path(OUTPUTS_FOLDER) / experiment_name
create_video(last_frame_index, Path(experiment_folder))

Asignando frames: 100%|██████████| 278/278 [00:05<00:00, 49.94it/s]


In [22]:
!cp $experiment_folder/video.mp4 temp.mp4
from IPython.display import Video
video_file = "temp.mp4"
Video(video_file)

In [10]:
!nvidia-smi

Tue Jun  1 02:53:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3060    Off  | 00000000:01:00.0  On |                  N/A |
| 38%   52C    P2    41W / 170W |  10542MiB / 12045MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Liberar memoria de la GPU
reset_kernel()



---


Notebook Original: https://colab.research.google.com/drive/1L8oL-vLJXVcRzCFbPwOoMkPKJ8-aYdPN